In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [7]:
import datajoint_utils as du
import datajoint as dj

In [25]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:20:56,013 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:20:56,308 - settings - Setting enable_python_native_blobs to True


# Inspecting Baylor Soma

In [66]:
(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()# & "status='error'"

In [64]:
len(dj.U("segment_id") & minnie.BaylorSegmentCentroid())

98

In [ ]:
schema

In [41]:
minnie.BaylorSegmentCentroid.delete()

100%|██████████| 29625/29625 [08:58<00:00, 54.99it/s]


[]

In [43]:
minnie.NeuronGliaNuclei.delete()

INFO - 2021-01-12 22:18:12,978 - connection - Transaction started


About to delete:
`microns_minnie65_02`.`neuron_glia_nuclei`: 29218 items
Proceed? [yes, No]: yes


INFO - 2021-01-12 22:18:15,074 - connection - Transaction committed and closed.


Committed.


In [44]:
schema.external['faces'].delete(delete_external_files=True)

100%|██████████| 58436/58436 [16:13<00:00, 60.00it/s]


[]

# Inspecting a soma extraction

In [45]:
import system_utils as su

In [46]:
glia_pieces= su.decompress_pickle("glia_pieces")
nuclei_pieces= su.decompress_pickle("nuclei_pieces")
total_soma_list = su.decompress_pickle("total_soma_list")

In [47]:
curr_mesh = du.fetch_segment_id_mesh(864691135403794158)

In [48]:
import neuron_visualizations as nviz

In [49]:
nviz.plot_objects(curr_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [51]:
nviz.plot_objects(meshes=total_soma_list)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Inspecting Decomposition

In [35]:
len(minnie.Decomposition())

80

In [16]:
(schema.jobs & "table_name='__decomposition'") & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,5f426ed37d3e1f147514d4ee7dc4dc44,error,=BLOB=,ValueError: cannot reshape array of size 1 into shape (3),=BLOB=,celiib@10.28.0.166,at-node39,29,30409,2021-01-12 01:21:20
__decomposition,b5223d0c5372809d087b36cd9373fba4,error,=BLOB=,ValueError: cannot reshape array of size 1 into shape (3),=BLOB=,celiib@10.28.0.153,at-node26,29,30267,2021-01-12 01:00:06
__decomposition,c127efa43819f30b1efbf507ca52e805,error,=BLOB=,Exception: missing labels was not resolved,=BLOB=,celiib@10.28.0.145,at-node18,29,30285,2021-01-12 01:10:56
__decomposition,e4a9ee9d6496f36782538e6edc19e941,error,=BLOB=,"Exception: The number of nodes in the concept graph and number of branches passed to it did not match len(curr_limb_concept_network.nodes())=206, len(curr_limb_divided_skeletons)= 207",=BLOB=,celiib@10.28.0.135,at-node8,29,30271,2021-01-12 00:35:36


# Inspecting Split Suggestions

In [26]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 18:21:13,220 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 18:21:13,512 - settings - Setting enable_python_native_blobs to True


In [34]:
len(minnie.NeuronSplitSuggestions())

76

In [32]:
(schema.jobs & "table_name='__neuron_split_suggestions'") & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__neuron_split_suggestions,087eb9c97f37e9b414e0649b41e5cac2,error,=BLOB=,Exception: base_node (40) and upstream_node (19) are not connected in the concept network,=BLOB=,celiib@10.21.18.111,71caefafdeec,26880,31121,2021-01-12 12:23:25
__neuron_split_suggestions,52a081c3587840e9f0981196ec9f1180,error,=BLOB=,Exception: base_node (74) and upstream_node (78) are not connected in the concept network,=BLOB=,celiib@10.21.18.111,71caefafdeec,26340,31116,2021-01-12 12:50:17
